# Improve Retrieval Augmented Generation using Semantic Chunking with Elastic, Anthropic Claude 3.7, Amazon Bedrock, Langchain and RAGAS

## Introduction

In this notebook, we will demonstrate how to leverage semantic chunking to enhance the performance of a Retrieval-Augmented Generation (RAG) solution built using Langchain, Anthropic Claude 3.7, Elastic, RAGAS,


#### Use case

Improving the relevance and coherence of retrieved contexts in a RAG application through semantic chunking.


#### Persona

As an analyst at Anycompany, Alice wants to improve the quality of the responses generated by the RAG solution. She has noticed that sometimes the retrieved contexts may lack coherence or contain irrelevant information, leading to suboptimal responses. Alice is seeking a technique to organize and filter the retrieved contexts more effectively, ensuring that only the most relevant and semantically related information is utilized for generating responses.

#### Implementation

To address this use case, we will incorporate semantic chunking into the RAG application. Semantic chunking is a process that groups related sentences or text units together based on their semantic meanings or relatedness, rather than simply splitting the text based on size or separators.

The implementation of semantic chunking in this RAG solution involves the following steps:

1. **Sentence Splitting**: The first step is to divide the text corpus into individual sentences, as sentences are considered the basic units of meaning.

2. **Sentence Pairing**: Neighboring sentences are paired together to capture the relationships between sequential sentences. This pairing involves a buffer size, which determines the number of preceding and leading sentences to combine.

3. **Embedding Generation**: The paired sentences are then converted into numerical representations called embeddings, which capture the underlying semantic meanings of the sentences.

4. **Similarity Calculation**: The cosine similarity between the embeddings of paired sentences is calculated to measure their semantic relatedness or distance.

5. **Visualization and Thresholding**: The cosine distances between sequential sentence embeddings are visualized to identify breakpoints or boundaries for creating semantic chunks. A threshold value, often determined using a percentile-based approach, is used to group sentences into semantic chunks based on their cosine distance or similarity.

By incorporating semantic chunking into the RAG solution, which utilizes the Anthropic Claude 3.7 Sonnet Foundation model, Elastic, Langchain and RAGAS, Alice can ensure that the retrieved contexts are semantically cohesive and highly relevant to the input question or query. This approach improves the overall quality and coherence of the responses generated by the RAG application.

#### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠


## Installation

To run this notebook you would need to install dependencies - boto3, botocore, elasticsearch and langchain.

In [2]:
%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install langchain --force-reinstall --quiet
%pip install langchain_aws --force-reinstall --quiet
%pip install langchain-elasticsearch --force-reinstall --quiet
%pip install elasticsearch --force-reinstall --quiet
%pip install pypdf --force-reinstall --quiet
%pip install ragas>0.1 --force-reinstall --quiet
%pip install langchain_experimental --force-reinstall --quiet

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.33.13 requires botocore==1.34.131, but you have botocore 1.35.76 which is incompatible.
sagemaker 2.224.1 requires attrs<24,>=23.1.0, but you have attrs 24.2.0 which is incompatible.
sagemaker 2.224.1 requires numpy<2.0,>=1.9.0, but you have numpy 2.1.3 which is incompatible.
sagemaker 2.224.1 requires protobuf<5.0,>=3.12, but you have protobuf 5.29.1 which is incompatible.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.6.0 which is incompatible.
langchain-aws 0.2.9 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.1.3 which is incompatible.
langchain 0.3.9 requires numpy<2,>=1.22.4; python_version < "3.12", but you have numpy 2.1.3 which is incompatible.
Note: you may need to restart the kerne

## Kernel Restart

Restart the kernel with the updated packages that are installed through the dependencies above

In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Setup 

Import the necessary libraries

In [4]:
import json
import os
import sys
import boto3
import botocore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrockConverse
from langchain_aws import AmazonKnowledgeBasesRetriever
from langchain_aws import BedrockEmbeddings
from botocore.client import Config
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_elasticsearch import ElasticsearchStore
from elasticsearch import Elasticsearch
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import RetrievalQA
from getpass import getpass
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader,TextLoader
from pathlib import Path
from datasets import Dataset
import pandas as pd

## Initialization

Initiate Bedrock Runtime and BedrockChat

In [ ]:
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')

modelId = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0' # change this to use a different version from the model provider
embeddingmodelId = 'amazon.titan-embed-text-v2:0' # change this to use a different embedding model

llm = ChatBedrockConverse(model_id=modelId, client=bedrock_client)
embeddings = BedrockEmbeddings(model_id=embeddingmodelId,client=bedrock_client)

## Read files from directory

Load all PDF files which are present in the directory

In [6]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [7]:
# publicly available dataset: https://github.com/CloudPak-Outcomes/Outcomes-Projects/blob/main/L4assets/watsonx.ai-Assets/Documents/state_of_the_union.txt
TMP_DIR = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'media/state_of_the_union.txt')
loader = TextLoader(TMP_DIR, encoding='ISO-8859-1')
documents = loader.load()

## Semantic Chunking

Semantic chunking is a process of dividing text into meaningful chunks or segments based on the semantic similarity or relatedness of sentences/text units, rather than simply splitting text based on size or separators. The key aspects of semantic chunking are:

1. Splitting the text into individual sentences as basic units.

2. Pairing neighboring sentences together to capture relationships between sequential sentences. The pairing involves a buffer size (e.g., 1 preceding and 1 leading sentence).

3. Converting the paired sentences into numerical embeddings, which capture the underlying semantic meanings of the sentences.

4. Calculating the cosine similarity between the embeddings of paired sentences to measure their semantic relatedness or distance.

5. Visualizing the cosine distances between sequential sentence embeddings to identify breakpoints or boundaries for creating semantic chunks.

6. Determining a threshold value (e.g., a percentile value like 95th percentile) to group sentences into semantic chunks based on their cosine distance/similarity.

The key advantage of semantic chunking is that it groups related sentences or text units together based on their semantic meanings, rather than chunking text without understanding the context or relatedness. This approach ensures that the resulting chunks contain semantically cohesive and related content, which can be beneficial for various NLP tasks, such as information extraction, summarization, or question answering.

## Breakpoints

This chunker works by determining when to break apart sentences. This is done by looking for differences in embeddings between any two sentences. When that difference is past some threshold, then they are split.

There are a few ways to determine what that threshold is.

### Percentile

The default way to split is based on percentile. In this method, all differences between sentences are calculated, and then any difference greater than the X percentile is split.

In [8]:
text_splitter = SemanticChunker(embeddings)
texts = text_splitter.split_documents(documents)

#### Visualize the split documents

In [9]:
texts

[Document(metadata={'source': '/root/aws-generativeai-partner-samples/elastic/samples/media/state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans. Last year COVID-19 kept us apart. This year we are finally together again. Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. With a duty to one another to the American people to the Constitution. And with an unwavering resolve that freedom will always triumph over tyranny. Six days ago, Russia\x92s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. He met the Ukrainian people. From President Zelenskyy to every Ukrainian, their fearlessness

#### Number of Documents after split

In [10]:
print(len(texts))

26


### Standard Deviation

In this method, any difference greater than X standard deviations is split.

In [11]:
text_splitter = SemanticChunker(embeddings, breakpoint_threshold_type="standard_deviation")
texts = text_splitter.split_documents(documents)

#### Visualize the split documents

In [12]:
texts

[Document(metadata={'source': '/root/aws-generativeai-partner-samples/elastic/samples/media/state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans. Last year COVID-19 kept us apart. This year we are finally together again. Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. With a duty to one another to the American people to the Constitution. And with an unwavering resolve that freedom will always triumph over tyranny. Six days ago, Russia\x92s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. He met the Ukrainian people. From President Zelenskyy to every Ukrainian, their fearlessness

#### Number of Documents after split

In [13]:
print(len(texts))

5


### Interquartile

In this method, the interquartile distance is used to split chunks.

In [14]:
text_splitter = SemanticChunker(embeddings, breakpoint_threshold_type="interquartile")
texts = text_splitter.split_documents(documents)

#### Visualize the split documents

In [15]:
texts

[Document(metadata={'source': '/root/aws-generativeai-partner-samples/elastic/samples/media/state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans. Last year COVID-19 kept us apart. This year we are finally together again. Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. With a duty to one another to the American people to the Constitution. And with an unwavering resolve that freedom will always triumph over tyranny. Six days ago, Russia\x92s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. He met the Ukrainian people. From President Zelenskyy to every Ukrainian, their fearlessness

#### Number of Documents after split

In [16]:
print(len(texts))

25


## Connect to Elasticsearch

We'll use the Cloud ID to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to [Cloud ID](https://cloud.elastic.co/deployments) and select your deployment.

We will use ElasticsearchStore to connect to our elastic cloud deployment. This would help create and index data easily. 

In [17]:
cloud_id = getpass("Elastic deployment Cloud ID: ")
cloud_api_key = getpass("Elastic deployment API Key: ")
index_name= "new-index-1"

vector_store = ElasticsearchStore(
        es_cloud_id=cloud_id,  
        index_name= index_name, 
        embedding=embeddings,
        es_api_key=cloud_api_key)

Elastic deployment Cloud ID:  ········
Elastic deployment API Key:  ········


## Index data into Elasticsearch and initialize retriever

Next, we will index data to elasticsearch using ElasticsearchStore.from_documents. We will use Cloud ID, Password and Index name values set in the Create cloud deployment step. We will set embedding to BedrockEmbeddings to embed the texts.

In [18]:
vectordb = vector_store.from_documents(
        texts, 
        embeddings,
        index_name=index_name,
        es_cloud_id=cloud_id,
        es_api_key=cloud_api_key
        )

retriever = vectordb.as_retriever()

## Model Invocation and Response Generation using RetrievalQA chain

Now that we have the passages stored in Elasticsearch and LLM is initialized, we can now ask a question to get the relevant passages.

In [19]:
query = "What did the president say about Ketanji Brown Jackson?"


prompt_template = """
    Human: You will be acting as an advisor named Poly created by the company Polymath. 
    Your goal is to give advice from the derived context. 
    site and who will be confused if you don't respond in the character of Poly.
    
    You should maintain a friendly customer service tone.

    Here is the document you should reference when answering the user: <context>{context}</context>

    Here are some important rules for the interaction:
    - Always stay in character, as Poly, a Machine Learning advisor on complex Mechanical systems
    - If you are unsure how to respond, say “Sorry, I didn’t understand that. Could you repeat the question?”
    - If someone asks something irrelevant, say, “Sorry, I don't know.”


    Here is the user’s question: <question> {question} </question>

    How do you respond to the user’s question?
    Think about your answer first before you respond. Put your response in <response></response> tags.
    Assistant: <response>"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context","question"])
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",retriever=retriever, return_source_documents=True, chain_type_kwargs={"prompt": prompt})
response = qa_chain.invoke(query)
print(response["result"])

According to the context, the president said the following about Ketanji Brown Jackson:

"One of our nation's top legal minds, who will continue Justice Breyer's legacy of excellence. A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she's been nominated, she's received a broad range of support - from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."

The president praised Ketanji Brown Jackson's legal expertise, experience as a public defender and litigator, her family background, and her ability to build consensus. He also noted that she has received support from law enforcement and judges across the political spectrum since her nomination to the Supreme Court.

</response>


## Preparing the Evaluation Data

As RAGAS aims to be a reference-free evaluation framework, the required preparations of the evaluation dataset are minimal. You will need to prepare `questions` and `references` pairs from which you can prepare the remaining information through inference as shown below. If you are not interested in the `context_recall` metric, you don’t need to provide the `references` information. In this case, all you need to prepare are the questions.

In [20]:
from ragas import SingleTurnSample, EvaluationDataset

questions = ["What did the president say about Ketanji Brown Jackson?"]

references = ["The president described Ketanji Brown a one of the nation's top legal minds"]

samples = []

for idx, query in enumerate(questions):
    samples.append(
        SingleTurnSample(
            user_input=query,
            retrieved_contexts=[docs.page_content for docs in retriever.invoke(query)],
            response=qa_chain.invoke(query)["result"],
            reference=references[idx]
        )
    )

dataset = EvaluationDataset(samples=samples)

## Evaluating the RAG application

First, import all the metrics you want to use from `ragas.metrics`. Then, you can use the `evaluate()` function and simply pass in the relevant metrics and the prepared dataset. Below is a brief description of the metrics

* **Faithfulness**: This measures the factual consistency of the generated answer against the given context. It is calculated from answer and retrieved context. The answer is scaled to (0,1) range. Higher the better.
* **Response Relevance**: The evaluation metric, Response Relevancy, focuses on assessing how pertinent the generated answer is to the given prompt. A lower score is assigned to answers that are incomplete or contain redundant information and higher scores indicate better relevancy. This metric is computed using the question, the context and the answer. Please note, that eventhough in practice the score will range between 0 and 1 most of the time, this is not mathematically guaranteed, due to the nature of the cosine similarity ranging from -1 to 1.
* **Context Precision**: Context Precision is a metric that evaluates whether all of the ground-truth relevant items present in the contexts are ranked higher or not. Ideally all the relevant chunks must appear at the top ranks. This metric is computed using the question, ground_truth and the contexts, with values ranging between 0 and 1, where higher scores indicate better precision.
* **Context Recall**: Context recall measures the extent to which the retrieved context aligns with the annotated answer, treated as the ground truth. It is computed based on the ground truth and the retrieved context, and the values range between 0 and 1, with higher values indicating better performance.
* **Context entities recall**: This metric gives the measure of recall of the retrieved context, based on the number of entities present in both ground_truths and contexts relative to the number of entities present in the ground_truths alone. Simply put, it is a measure of what fraction of entities are recalled from ground_truths. This metric is useful in fact-based use cases like tourism help desk, historical QA, etc. This metric can help evaluate the retrieval mechanism for entities, based on comparison with entities present in ground_truths, because in cases where entities matter, we need the contexts which cover them.
* **Answer Semantic Similarity**: The concept of Answer Semantic Similarity pertains to the assessment of the semantic resemblance between the generated answer and the ground truth. This evaluation is based on the ground truth and the answer, with values falling within the range of 0 to 1. A higher score signifies a better alignment between the generated answer and the ground truth.
* **Answer Correctness**: The assessment of Answer Correctness involves gauging the accuracy of the generated answer when compared to the ground truth. This evaluation relies on the ground truth and the answer, with scores ranging from 0 to 1. A higher score indicates a closer alignment between the generated answer and the ground truth, signifying better correctness. Answer correctness encompasses two critical aspects: semantic similarity between the generated answer and the ground truth, as well as factual similarity. These aspects are combined using a weighted scheme to formulate the answer correctness score. Users also have the option to employ a ‘threshold’ value to round the resulting score to binary, if desired.
* **Aspect Critique**: This is designed to assess submissions based on predefined aspects such as harmlessness, maliciousness, coherence, and conciseness. The output of aspect critiques is binary, indicating whether the submission aligns with the defined aspect or not. This evaluation is performed using the ‘answer’ as input.

In [21]:
from ragas.metrics import (
        LLMContextRecall, 
        Faithfulness, 
        LLMContextPrecisionWithReference, 
        AnswerCorrectness, 
        ResponseRelevancy, 
        SemanticSimilarity, 
        AspectCritic
    )
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas import evaluate 

#You can also choose a different model for evaluation
llm_for_evaluation = LangchainLLMWrapper(ChatBedrockConverse(model_id=modelId, client=bedrock_client))
bedrock_embeddings = LangchainEmbeddingsWrapper(BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0",client=bedrock_client))

#specify the metrics here
metrics = [
    LLMContextRecall(llm=llm_for_evaluation), 
    LLMContextPrecisionWithReference(llm=llm_for_evaluation),
    AnswerCorrectness(llm=llm_for_evaluation, embeddings=bedrock_embeddings), 
    ResponseRelevancy(llm=llm_for_evaluation, embeddings=bedrock_embeddings),
    Faithfulness(llm=llm_for_evaluation),
    SemanticSimilarity(embeddings=bedrock_embeddings),
    AspectCritic(name="harmfulness", 
         definition="Does the submission cause or have the potential to cause harm to individuals, groups, or society at large?", 
         llm=llm_for_evaluation
        ),
    AspectCritic(name="maliciousness", 
                 definition="Is the submission intended to harm, deceive, or exploit users?", 
                 llm=llm_for_evaluation
                ),
    AspectCritic(name="coherence", 
             definition="Is the submission logical, relevant, and informative along with clear structure?", 
             llm=llm_for_evaluation
            ),
    AspectCritic(name="conciseness", 
         definition="Is the submission brief, direct, and avoids unnecessary wordiness while conveying intended meaning?", 
         llm=llm_for_evaluation
        )
    ]

result = evaluate(
    dataset = dataset, 
    metrics=metrics
)

df = result.to_pandas()

df.style.set_properties(**{'text-align': 'left'}).set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])
pd.options.display.max_colwidth = 8000
df

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

user_input  \
0  What did the president say about Ketanji Brown Jackson?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

## Delete Elasticsearch Index

Delete the Elasticsearch index

In [22]:
es = Elasticsearch(cloud_id=cloud_id, api_key=cloud_api_key)
es.options(ignore_status=[400,404]).indices.delete(index=index_name)

ObjectApiResponse({'acknowledged': True})

## Conclusion
You have now experimented with reranking to improve the output of a RAG Application.

### Take aways
- Adapt this notebook to experiment with different Claude 3 models available through Amazon Bedrock. 
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You